<!-- TABS -->
# Build image embedding model
#### Construct a neural network architecture to project high-dimensional image data into a lower-dimensional, dense vector representation (embedding) that preserves relevant semantic and visual information within a learned latent space.

In [1]:
# <tab: torchvision>
# !pip install torch
# !pip install torchvision

import torch
import torchvision.models as models
from torchvision import transforms
from superduperdb.ext.torch import TorchModel

class TorchVisionEmbedding:
    def __init__(self):
        # Load the pre-trained ResNet-18 model
        self.resnet = models.resnet18(pretrained=True)
        
        # Set the model to evaluation mode
        self.resnet.eval()
        
    def preprocess(self, image):
        # Preprocess the image
        preprocess = preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        tensor_image = preprocess(image)
        return tensor_image
        
model = TorchVisionEmbedding()
superdupermodel = TorchModel(identifier='torchvision', object=model.resnet, preprocess=model.preprocess)

/Users/kartiksharma/Work/superduperdb/code/superduperdb/.venv_11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kartiksharma/Work/superduperdb/code/superduperdb/.venv_11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [1]:
# <tab: CLIP-multimodal>
# !pip install torch
# !pip install torchvision
# !pip install git+https://github.com/openai/CLIP.git
import torch
import clip
from torchvision import transforms
from superduperdb.ext.torch import TorchModel

class CLIPVisionEmbedding:
    def __init__(self):
        # Load the CLIP model
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model, self.preprocess = clip.load("RN50", device=self.device)
        
    def preprocess(self, image):
        # Load and preprocess the image
        image = self.preprocess(image).unsqueeze(0).to(self.device)
        return image
        
model = CLIPVisionEmbedding()
superdupermodel = TorchModel(identifier='torchvision', object=model.model, preprocess=model.preprocess, forward_method='encode_image')

In [7]:
# <tab: HuggingFace (ViT) >
# !pip install transformers
# !pip install torch
# !pip install torchvision
import torch
from transformers import AutoImageProcessor, AutoModel, AutoFeatureExtractor
import torchvision.transforms as T
from superduperdb.ext.torch import TorchModel


class HuggingFaceEmbeddings(torch.nn.Module):
    def __init__(self):
        super().__init__()
        model_ckpt = "nateraw/vit-base-beans"
        processor = AutoImageProcessor.from_pretrained(model_ckpt)
        self.extractor = AutoFeatureExtractor.from_pretrained(model_ckpt)
        self.model = AutoModel.from_pretrained(model_ckpt)

    def forward(self, x):
        return self.model(pixel_values=x).last_hidden_state[:, 0].cpu()
        
        
class Preprocessor:
    def __init__(self, extractor):
        self.device = 'cpu'
        # Data transformation chain.
        self.transformation_chain = T.Compose(
            [
                # We first resize the input image to 256x256 and then we take center crop.
                T.Resize(int((256 / 224) * extractor.size["height"])),
                T.CenterCrop(extractor.size["height"]),
                T.ToTensor(),
                T.Normalize(mean=extractor.image_mean, std=extractor.image_std),
            ]
        )
    def __call__(self, image):
        return self.transformation_chain(image).to(self.device)


    
model = HuggingFaceEmbeddings()
superdupermodel = TorchModel(identifier='torchvision', object=model, preprocess=Preprocessor(model.extractor))

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.
Some weights of ViTModel were not initialized from the model checkpoint at nateraw/vit-base-beans and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
# <testing>
import numpy as np
from PIL import Image

embeddings = superdupermodel.predict_one(Image.fromarray(np.ones((256,256,3)).astype(np.uint8)))

 2024-Mar-26 18:17:39.58| INFO     | kartiks-MacBook-Air.local| superduperdb.components.component:344  | Initializing TorchModel : torchvision
 2024-Mar-26 18:17:39.61| INFO     | kartiks-MacBook-Air.local| superduperdb.components.component:347  | Initialized  TorchModel : torchvision successfully
